In [2]:
from scTrace.pipeline import *

E:\Software\Anaconda\installation\envs\scTrace_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
runLabel = "PC9"
savePath = "D:/Lineage-data/Nature21-Watermelon/Python-data/"
dataPath = "D:/Lineage-data/Nature21-Watermelon/R-data/"

In [3]:
adata = sc.read_10x_mtx(
    dataPath + 'PC9/filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',    # use gene symbols for the variable names (variables-axis index)
    make_unique = True, cache=False)

In [4]:
cellAnno = pd.read_csv(dataPath + 'PC9/cellAnno.txt', sep = '\t')

In [5]:
adata.obs = cellAnno

In [6]:
adata

AnnData object with n_obs × n_vars = 56419 × 22166
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'sample_id', 'time_point', 'sample_name', 'sample_type', 'lineage_barcode', 'majority_fate'
    var: 'gene_ids', 'feature_types'

In [7]:
adata.obs['time_point'].unique()

array([ 0,  3,  7, 14])

In [ ]:
def getScObjects(scobj, item, values, n_neighbors = 10, n_pcs = 40):
    data_split = [scobj[scobj.obs[item] == v,:] for v in values]
    data_split = [preProcess(d) for d in data_split]
    comm_hvgs = set([g for d in data_split for g in d.var.index[d.var.highly_variable]])
    
    for di in range(len(data_split)):
        data_split[di].var.highly_variable = [gene in comm_hvgs for gene in data_split[di].var.index]
        data_split[di] = runScanpy(data_split[di], n_neighbors, n_pcs, regress=False)
    
    return(data_split)

In [17]:
adata.obs['time_point'].unique()

array([ 0,  3,  7, 14])

In [19]:
data0, data3, data7, data14 = getScObjects(adata, item = 'time_point', values = [0,3,7,14], n_neighbors = 10, n_pcs = 40)

/home/glab/wguo/softwares/anaconda3/envs/gwb-r40/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [30]:
data0.write(savePath + 'D0_scobj.h5ad')
data3.write(savePath + 'D3_scobj.h5ad')
data7.write(savePath + 'D7_scobj.h5ad')
data14.write(savePath + 'D14_scobj.h5ad')